# US Golf Course Information Kiosk

Detailed information on golf courses with green fees range and tees, this complete golf courses database has 15,606 records across 7,891 cities over 53 states in the United States. Each record is comprised of address, street, phone number, zip code, hole, architect, year built, public/private, guest policy, credit card, golf season, range, rental club, pro in house, metal spikes okay, weekday, weekend, tee time welcomed, rental cart available, championship par/yards/slope/USGA, middle par/yards/slope/USGA and forward par/yards/slope/USGA

https://www.usabledatabases.com/database/golf-courses-in-us/

## Step 1:  Extract, clean and transform the input dataset

Before running this extract, clean and transform process, make sure that you update the config.py file with a Google Map API Key as gkey 


In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

# CSV files
course = "data/course.csv"
city = "data/city.csv"
state = "data/state.csv"

### Longitude and Latitude for mapping golf courses using street address

In [ ]:
# Create the data frame - golf courses
course_df = pd.read_csv(course, 
                        delimiter=',', 
                        skipinitialspace=True, 
                        dtype={"zip_code":object, "hole":object })

# Add columns for longitude and latitude
course_df["lng"] = ""
course_df["lat"] = ""

golf_courses = course_df.rename(columns={"id": "course_id",
                                         "title": "course"})

# Replace &amp; with &
golf_courses = golf_courses.replace('&amp;','&', regex=True)

golf_courses

In [ ]:
# Exclude all golf courses that do not have championship, middle, and forward tee information
nan_value = float("NaN")
golf_courses.replace("", nan_value, inplace=True)
golf_courses.dropna(subset = ["championship_par"], inplace=True)
golf_courses.dropna(subset = ["middle_par"], inplace=True)
golf_courses.dropna(subset = ["forward_par"], inplace=True)
golf_courses.dropna(subset = ["championship_slope"], inplace=True)
golf_courses.dropna(subset = ["middle_slope"], inplace=True)
golf_courses.dropna(subset = ["forward_slope"], inplace=True)
golf_courses.dropna(subset = ["championship_usga"], inplace=True)
golf_courses.dropna(subset = ["middle_usga"], inplace=True)
golf_courses.dropna(subset = ["forward_usga"], inplace=True)

# Exclude all golf courses that do not have hole, public-private, season information
golf_courses.dropna(subset = ["hole"], inplace=True)
golf_courses.dropna(subset = ["public_private"], inplace=True)
golf_courses.dropna(subset = ["golf_season"], inplace=True)

golf_courses

### Checking for duplicate rows of golf courses, cities, states

In [ ]:
# Checking for duplicate rows in golf courses
duplicateRow = golf_courses[golf_courses.duplicated()]
duplicateRow

In [ ]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "56001") & (golf_courses["zip_code"] <= "56007"))]

subset_df

In [ ]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "55001") & (golf_courses["zip_code"] <= "56763"))]

subset_df

In [ ]:
# Create a subset data frame - variety of golf courses
bool_series = golf_courses["zip_code"].str.startswith("010", na = False) 
df1 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("6000", na = False) 
df2 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("5600", na = False) 
df3 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("920", na = False) 
df4 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("2001", na = False) 
df5 = golf_courses[bool_series]

subset_df = df1.append(df2, ignore_index = True)
subset_df = subset_df.append(df3, ignore_index = True)
subset_df = subset_df.append(df4, ignore_index = True)
subset_df = subset_df.append(df5, ignore_index = True)
subset_df

In [ ]:
# Create a subset data frame - All golf courses
subset_df = golf_courses

subset_df

In [ ]:
# Create the data frame - golf cities
city_df = pd.read_csv(city, delimiter=',', skipinitialspace=True)

golf_cities = city_df.rename(columns={"id": "city_id", 
                                      "title": "city_title", 
                                      "count": "city_count", 
                                      "slug": "city_slug"})
golf_cities

In [ ]:
# Checking for duplicate rows in golf cities
duplicateRow = golf_cities[golf_cities.duplicated()]
duplicateRow

In [ ]:
# Create the data frame - golf states
state_df = pd.read_csv(state, delimiter=',', skipinitialspace=True)

golf_states = state_df.rename(columns={"id": "state_id", 
                                       "title": "state", 
                                       "count": "state_count", 
                                       "slug": "state_slug"})
golf_states

In [ ]:
# Checking for duplicate rows in golf states
duplicateRow = golf_states[golf_states.duplicated()]
duplicateRow

### Merge the data frames to create one complete dataset

In [ ]:
# Merge the city and state dataframes using a left join
city_state_df = pd.merge(golf_cities, golf_states, on="state_id", how="left")
city_state_df

In [ ]:
# Merge the city_state dataframe with the golf course dataframe using a left join
golf_citystate = pd.merge(subset_df, city_state_df, on="city_id" ,how="left")
golf_citystate

In [ ]:
regex = r'(?P<c_city>[^,]+)\s*,\s*'
golf_address = golf_citystate.join(golf_citystate["address"].str.extract(regex))
golf_address

In [ ]:
# Finding the best city to use
golf_address["city"] = golf_address.c_city.combine_first(golf_address.city_title)

golf_df = golf_address
golf_df

### All golf courses along the Mississippi River

In [ ]:
# Golf courses along the Mississippi River - 
# Minnesota, Wisconsin, Iowa, Illinois, Missouri, Kentucky, Tennessee, Arkansas, Mississippi, Louisiana
mask = golf_df["state_id"].isin([24, 52, 16, 14, 26, 18, 44, 4, 25, 19])
MissRiver_golf = golf_df[mask]
MissRiver_golf

### All available information for the golf kiosk

In [ ]:
# Select the golf course columns needed for course longitude, latitude
golf_course_df = MissRiver_golf[["course_id",
                                 "course",
                                 "city",
                                 "state",
                                 "street",
                                 "zip_code",
                                 "lng",
                                 "lat",
                                 "hole",
                                 "public_private",
                                 "golf_season"]]
                     
# Display the new dataframe
golf_course_df

In [ ]:
# Select the golf course columns needed for golf kiosk - general information
course_info = golf_df[["course_id",
                       "phone",
                       "architect",
                       "year_built",
                       "guest_policy",
                       "credit_card"]]
                     
# Display the new dataframe
course_info

In [ ]:
# Select the golf course columns needed for golf kiosk - course details
course_details = golf_df[["course_id",
                          "range",
                          "rental_club",
                          "pro_in_House",
                          "metal_spikes_okay",
                          "weekday",
                          "weekend",
                          "tee_time_welcomed",
                          "rental_cart_available"]]
                     
# Display the new dataframe
course_details

In [ ]:
# Select the golf course columns needed for golf kiosk - championship tees
championship_tees = golf_df[["course_id", 
                            "championship_par", 
                            "championship_yards", 
                            "championship_slope", 
                            "championship_usga"]]
                     
# Display the new dataframe
championship_tees

In [ ]:
# Select the golf course columns needed for golf kiosk - middle tees
middle_tees = golf_df[["course_id",
                       "middle_par",
                       "middle_yards",
                       "middle_slope",
                       "middle_usga"]]
                     
# Display the new dataframe
middle_tees

In [ ]:
# Select the golf course columns needed for golf kiosk - forward tees
forward_tees = golf_df[["course_id",
                        "forward_par",
                        "forward_yards",
                        "forward_slope",
                        "forward_usga"]]
                     
# Display the new dataframe
forward_tees

In [ ]:
golf_course_df

### Getting the longitude and latitude for each golf course based on street address first, otherwise zip code

In [ ]:
# Getting the longitude and latitude for each golf course
count = 0

# Use the street, city, state to get lng and lat
for index, row in golf_course_df.iterrows():
    # get street, city, state from golf_course_df
    street = row["street"]
    city = row["city"]
    state = row["state"]
    postal_code = row["zip_code"]
    
    # Getting lat/lng
    query_address_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={street},+{city},+{state}&key={gkey}"
    # print(f"url = {query_address_url}")
        
    query_zip_url = f"https://maps.googleapis.com/maps/api/geocode/json?components=postal_code:{postal_code}&key={gkey}"
    # print(f"url = {query_zip_url}")

    response = requests.get(query_address_url).json()
    # print(f"response = {response}")
    
    count = count + 1
    # print(f"#{count} - index = {index}")
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        golf_course_df.loc[index, "lat"] = response["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        response_zip = requests.get(query_zip_url).json()
        # print(f"response_zip = {response_zip}")golf_course_df.loc[index, "lat"] = response_zip["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response_zip["results"][0]["geometry"]["location"]["lng"]
        print("Using zip code for lng, lat.")
        

In [ ]:
# Preview course subset
golf_course_df

In [ ]:
# Create a dataframe to export
golf_courses = golf_course_df[["course_id",
                               "course",
                               "city",
                               "state",
                               "street",
                               "zip_code",
                               "lng",
                               "lat",
                               "hole",
                               "public_private", 
                               "golf_season"]]
                     
# Display the new dataframe
golf_courses

### Stripping out the beginning and ending months of the golf season for each course

In [ ]:
# Golf season begin and end months
for index, row in golf_courses.iterrows():
    str_open = row["golf_season"][0:5]
    beg_mnth = row["golf_season"][6:7]
    str_closed = row["golf_season"][11:18]
    end_mnth = row["golf_season"][19:21]
    end_day = row["golf_season"][22:24]

    if str_closed == " Closed" :
        beg_mnth = row["golf_courses"][6:8]
        str_closed = row["golf_courses"][12:19]
        end_mnth = row["golf_courses"][20:22]
        end_day = row["golf_courses"][22:24]
   
    if str_closed == "Closed:" :
        if end_day == "01":
            if end_mnth == "1/":
                end_mnth = "12"
            elif end_mnth == "4/":
                end_mnth = "3"
            elif end_mnth == "5/":
                end_mnth = "4"
            elif end_mnth == "6/":
                end_mnth = "5"
            elif end_mnth == "7/":
                end_mnth = "6"
            elif end_mnth == "8/":
                end_mnth = "7"
            elif end_mnth == "9/":
                end_mnth = "7"          
            elif end_mnth == "10":
                end_mnth = "9"
            elif end_mnth == "11":
                end_mnth = "10"             
            elif end_mnth == "12":
                end_mnth = "11"
        else:
            if end_mnth == "1/":
                end_mnth = "01"
            elif end_mnth == "4/":
                end_mnth = "04"
            elif end_mnth == "5/":
                end_mnth = "05"
            elif end_mnth == "6/":
                end_mnth = "06"
            elif end_mnth == "7/":
                end_mnth = "07"
            elif end_mnth == "8/":
                end_mnth = "08"
            elif end_mnth == "9/":
                end_mnth = "09"          
    else:
        end_mnth = "12"

   
    if str_open == "Open:" :
        if beg_mnth not in ["2", "3", "4", "5", "6", "7", "10", "11", "12"]:
            beg_mnth = "1"
    else:
        beg_mnth = "1"      

    golf_courses.loc[index,"beg_mnth"] = beg_mnth
    golf_courses.loc[index,"end_mnth"] = end_mnth
    
golf_courses

### Bringing all the information for the golf kiosk together

In [ ]:
# Merge the tee information with the golf courses dataframe using a left join
golf_tees1 = pd.merge(golf_courses, championship_tees, on="course_id" ,how="left")
golf_tees2 = pd.merge(golf_tees1, middle_tees, on="course_id" ,how="left")
golf_tees3 = pd.merge(golf_tees2, forward_tees, on="course_id" ,how="left")
golf_tees3

In [ ]:
# Export the golf course data to csv
golf_tees3.to_csv("results/MissRiver_golf.csv", index=False, header=True)

In [ ]:
# Merge more details with the golf courses dataframe using a left join
golf_info = pd.merge(golf_tees3, course_info, on="course_id" ,how="left")
golf_info

In [ ]:
# Merge more details with the golf courses dataframe using a left join
golf_details = pd.merge(golf_info, course_details, on="course_id" ,how="left")
golf_details

In [ ]:
# Export the golf course data to csv
golf_details.to_csv("results/MissRiver_golf_details.csv", index=False, header=True)

## Step 2:  To load the golf details into a PostgreSQL database, run golf_db_load.ipynb

Before running the load process, follow these steps:

1. create a new database in ProstgreSQL and name it Golf

2. run schema.sql to create the table in the new database

3. use a config.py file to store username and password for the new database you just created - it should look something like this

        username = "postgres"
        password = "mypassword"
